In [12]:
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

 # PTT NBA 板
url = "https://www.ptt.cc/bbs/Stock/index.html"

# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 

# 建立 list 來放置列表資訊
list_posts = []

In [13]:
# 第一個 <a></a>
print(soup.find("a"))

# 全部 <a></a>，此時回傳 list
print(soup.find_all("a"))

# 指定 list 某個元素的 html
print(soup.find_all("a")[1])

<a href="/bbs/" id="logo">批踢踢實業坊</a>
[<a href="/bbs/" id="logo">批踢踢實業坊</a>, <a class="board" href="/bbs/Stock/index.html"><span class="board-label">看板 </span>Stock</a>, <a class="right small" href="/about.html">關於我們</a>, <a class="right small" href="/contact.html">聯絡資訊</a>, <a class="btn selected" href="/bbs/Stock/index.html">看板</a>, <a class="btn" href="/man/Stock/index.html">精華區</a>, <a class="btn wide" href="/bbs/Stock/index1.html">最舊</a>, <a class="btn wide" href="/bbs/Stock/index7613.html">‹ 上頁</a>, <a class="btn wide disabled">下頁 ›</a>, <a class="btn wide" href="/bbs/Stock/index.html">最新</a>, <a href="/bbs/Stock/M.1722751713.A.CF4.html">[標的] 00929 與 00936</a>, <a href="/bbs/Stock/search?q=thread%3A%5B%E6%A8%99%E7%9A%84%5D+00929+%E8%88%87+00936">搜尋同標題文章</a>, <a href="/bbs/Stock/search?q=author%3Asiiaagl0">搜尋看板內 siiaagl0 的文章</a>, <a href="/bbs/Stock/M.1722754356.A.8B1.html">Re: [請益] 美國開啟無限QE的後果</a>, <a href="/bbs/Stock/search?q=thread%3A%5B%E8%AB%8B%E7%9B%8A%5D+%E7%BE%8E%E5%9C%8B%E

In [14]:
# 取得 id 為 logo 的元素
logo = soup.find(id="logo")
print(logo)

<a href="/bbs/" id="logo">批踢踢實業坊</a>


In [ ]:
for a in soup.select('a'):
    if a.has_attr('href'):
        print(a['href']) # a.get("href")
    else:
        print("=" * 50)
        print(f"連結[{a.get_text()}] 沒有 href 屬性")
        print("=" * 50)

In [17]:
# 清空放置列表資訊的變數
list_posts.clear()

# 取得 列表 的文字與超連結
for a in soup.select('div.r-ent div.title a[href]'):
    print(a.get_text())
    print(a['href']) # 或是 a.get('href') 
    
    # 加入列表資訊
    list_posts.append({
        'title': a.get_text(),
        'link': 'https://www.ptt.cc' + a['href']
    })

[標的] 00929 與 00936
/bbs/Stock/M.1722751713.A.CF4.html
Re: [請益] 美國開啟無限QE的後果
/bbs/Stock/M.1722754356.A.8B1.html
Re: [請益] 正二教的各位還好嗎
/bbs/Stock/M.1722754799.A.4D8.html
Re: [新聞] 輝達最新AI晶片驚傳設計瑕疵　外媒爆出
/bbs/Stock/M.1722755046.A.50A.html
[公告] 股票板板規 v4.6 (2024/07/02 修正)
/bbs/Stock/M.1719872231.A.9BA.html
[閒聊] 2024/08/02 盤後閒聊
/bbs/Stock/M.1722578402.A.E90.html


In [20]:
# 走訪每一個 a link，整合網頁內文
for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    soup_ = bs(res_.text, "lxml")

    # 去掉 div.article-metaline (作者、標題、時間…等)
    for div in soup_.select('div[class^="article-metaline"]'):
        div.decompose()

    for span in soup_.select('span[class="f2"]'):
        span.decompose()
        
    # 去掉 div.push (推文: 推、→、噓) (判斷元素是否存在)
    if len( soup_.select('div.push') ) > 0:
        for div in soup_.select('div.push'):
            div.decompose()
    
    # 取得實際需要的內容 (類似 JavaScript 的 innerHTML)
    html = soup_.select_one('div#main-content').decode_contents()
    # html = str(soup_.select_one('div#main-content')) # 類似 JavaScript outerHTML
    
    # 預覽列表和內文
    print(obj['title'])
    print(obj['link'])
    print(html)
    print("=" * 50)
    
    # 整合到列表資訊的變數當中
    list_posts[index]['html'] = html


# 預覽所有結果
pprint(list_posts)

[標的] 00929 與 00936
https://www.ptt.cc/bbs/Stock/M.1722751713.A.CF4.html

標的：
<span class="f0 hl">(例 2330.TW 台積電)</span>
<span class="hl">00929 與 00936</span>

分類：多/空/討論/心得
<span class="f0 hl">(請選擇並刪除無關分類)</span>
<span class="hl">討論</span>

分析/正文：
近日大盤跌跌不休

現在手邊有一點閒錢
敢問各大法人是要買00929比較好還是00936?

00929 外資大多隔日衝，長老喜愛，
7/31日 買6587張
8/1日 買2256張
8/2日 買20781張
3日下來買共29624張

00936
持股跟別的etf比較不一樣
配息還算可以

以這兩支ETF請指點迷津

--
感謝指教
!?
是的!兩支都是0.58%
感謝指教
感謝指教
了解
感謝指教
感謝指教
現在才第一顆紅燈要藍燈...有點久遠
感謝指教
!!!  感謝指教
板上很熱門
感謝指教
感謝指教  713 +3
感謝指教
了解  已納入參考
感謝指教
了解
感謝指教
感謝指教
看情況?

Re: [請益] 美國開啟無限QE的後果
https://www.ptt.cc/bbs/Stock/M.1722754356.A.8B1.html

<span class="f6">: <a href="http://www.apiaa.org.tw/information_show.php?pid=5&amp;sid=14&amp;id=1192" rel="noreferrer noopener nofollow" target="_blank">http://www.apiaa.org.tw/information_show.php?pid=5&amp;sid=14&amp;id=1192</a>
</span><span class="f6">: 其實早在2020-2022就有討論過美國開啟了無限QE這個潘朵拉盒子的後果
</span><span class="f6">: 但其實沒有人知道答案會是什麼
</span><span class="f6">: 當時還會有點怕，但實